In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define our data
years_exp = [1, 3, 4, 6, 8, 10]
salary = [40000, 50000, 55000, 60000, 75000, 85000]

# Create scatter plot
plt.scatter(years_exp, salary, color='red')

# Overplot a sine wave
xwave = np.linspace(0, 10, 100)  # 100 evenly spaced values from 0 to 10
ywave = np.sin(xwave) * 10000 + 60000  # Computes the corresponding sine values and adjust the amplitude and offset

plt.plot(xwave, ywave)

# Add title and labels
plt.title('Salary vs. Years of Experience with Sine Wave')
plt.xlabel('Years of Experience')
plt.ylabel('Salary')

# Show the plot
plt.show()

In [ ]:
import wave
import numpy as np
import matplotlib.pyplot as plt

def load_wave_as_int16(filename):
    with wave.open(filename, 'rb') as w:
        params = w.getparams()
        nchannels, sampwidth, framerate, nframes = params[:4]
        print(params)
        str_data = w.readframes(nframes)
        wave_data = np.frombuffer(str_data, dtype=np.short)
        wave_data.shape = -1, nchannels
        wave_data = wave_data.T
        return wave_data, framerate

# Change the filename to your file's path
# acc = load_wave_as_int16('output/output_voice/accompaniment.wav')
# voc = load_wave_as_int16('output/output_voice/vocals.wav')


data0, rate0 = load_wave_as_int16('output/output_voice/accompaniment.wav')

data1, rate1 = load_wave_as_int16('output/output_voice/vocals.wav')

# Plot the data
time = np.linspace(0., len(data0[0]) / rate0, len(data0[0]))
# plt.figure(figsize=(10, 4))
plt.plot(time, (data0[0]), label='Channel 1')
# plt.plot(time, (data0[0] - data1[0]), label='Channel 1')
# plt.show()

In [ ]:
import soundfile as sf
import resampy
import numpy as np
import matplotlib.pyplot as plt

target_sample_rate = 16000  # Set the sample rate to which you want to downsample.

# Read the audio files and resample them.
signal1, sample_rate1 = sf.read("output4.wav")
signal1_resampled = resampy.resample(signal1, sample_rate1, target_sample_rate)

signal2, sample_rate2 = sf.read("output4/output4/vocals.wav")

signal2_resampled = resampy.resample(signal2[:,0], sample_rate2, target_sample_rate)

# Compute the difference
difference = signal1_resampled - signal2_resampled

# Generate time axis values
time1 = np.linspace(0., len(signal1_resampled)/target_sample_rate, len(signal1_resampled))
time2 = np.linspace(0., len(signal2_resampled)/target_sample_rate, len(signal2_resampled))
time_diff = np.linspace(0., len(difference)/target_sample_rate, len(difference))

plt.figure()

# Plot signal 1
plt.subplot(3, 1, 1)
plt.plot(time1, signal1_resampled)
plt.title("file1.wav")

# Plot signal 2
plt.subplot(3, 1, 2)
plt.plot(time2, signal2_resampled)
plt.title("file2.wav")

# Plot the difference
plt.subplot(3, 1, 3)
plt.plot(time_diff, difference)
plt.title("difference")

plt.tight_layout()
plt.show()

In [ ]:
from pydub import AudioSegment
import os

def convert_ogg_to_wav(ogg_file_path, wav_file_path):
    audio = AudioSegment.from_ogg(ogg_file_path)
    audio.export(wav_file_path, format='wav')
    print(f'Successfully converted {ogg_file_path} to {wav_file_path}')

# put the path to your ogg file here
ogg_file_path = 'output_voice.ogg'  
# put the desired path to your wav file here
wav_file_path = 'output_voice.wav'

convert_ogg_to_wav(ogg_file_path, wav_file_path)


In [ ]:
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt

# Assuming you have two signals signal1 and signal2
signal1 = np.random.normal(size = 5000)
signal2 = np.random.normal(size = 5000)

# Apply the cross correlation
corr = scipy.signal.correlate(signal1, signal2, mode='full')

# Get the lag/position of the maximum correlation
lag_max_corr = np.argmax(corr) - (len(signal1) - 1)

# Plot the cross correlation
plt.figure(figsize=(14, 5))
plt.title('Cross-correlation')
plt.plot(corr/)
plt.show()

print(f"The maximum correlation is at lag {lag_max_corr}")

In [ ]:
from scipy.fft import fft
import numpy as np
import scipy.signal as sg
import soundfile as sf
import matplotlib.pyplot as plt

def load_audio(filepath, sr=None, mono=True, dtype='float32'):
    
    x, fs = sf.read(filepath)

    if mono and len(x.shape)>1:
        x = np.mean(x, axis = 1)
    if sr:
        x = sg.resample_poly(x, sr, fs)
        fs = sr 
    x = x.astype(dtype)

    return x, fs

# Original wav track after spleeter
x, fr = load_audio('output4.wav')
x1, fr1 = load_audio('output4/output4/vocals.wav')
x2, fr2 = load_audio('output4/output4/accompaniment.wav')


plt.figure()

plt.subplot(3, 1, 1)
plt.plot(np.linspace(0., len(x) / fr, len(x)), x)
plt.title("1")

# Plot signal 2
plt.subplot(3, 1, 2)
plt.plot(np.linspace(0., len(x1) / fr1, len(x1)), x1)
plt.title("2")


diff = x - x1

# Plot the difference
plt.subplot(3, 1, 3)
plt.plot(np.linspace(0., len(x1) / fr1, len(x1)), diff)
plt.title("difference")




print('np.correlate:', scipy.signal.correlate(x, x1))
print('fft diff:', max(np.abs(fft(x)) - np.abs(fft(x1))))
print('mse', np.mean((x - x1)**2))

# assuming signal1 and signal2 are your two signals
norm_cross_corr = scipy.signal.correlate(x, x1, mode='valid')
norm = np.sqrt(np.sum(x ** 2) * np.sum(x1 ** 2))
if norm != 0:
    norm_cross_corr /= norm
    
# max of cross-correlation will give a measure of similarity
similarity = np.max(np.abs(norm_cross_corr))
print('similarity:', similarity)

# time_diff = np.linspace(0., len(difference)/target_sample_rate, len(difference))

#plt.tight_layout()
plt.show()

In [ ]:
from spleeter.separator import Separator
import numpy as np
import scipy.signal as sg
import soundfile as sf

# Using embedded configuration.
separator = Separator('spleeter:2stems')

# # Using custom configuration file.

def load_audio(filepath, sr=None, mono=True, dtype='float8'):
    
    x, fs = sf.read(filepath)

    if mono and len(x.shape)>1:
        x = np.mean(x, axis = 1)
    if sr:
        x = sg.resample_poly(x, sr, fs)
        fs = sr 
    x = x.astype(dtype)

    return x, fs

# # Original wav track after spleeter
x, fr = load_audio('output4.wav')
x = np.array(x,ndmin = 2)
x = np.transpose(x)
np.shape(x)

import time


t = time.time()
res = separator.separate(x)
print(time.time() - t)
np.shape(res['vocals'][:,0])
x = np.transpose(x)[0]
x1 = res['vocals'][:,0]
print(np.shape(x), np.shape(x1))
norm_cross_corr = scipy.signal.correlate(x, x1, mode='valid')
norm = np.sqrt(np.sum(x ** 2) * np.sum(x1 ** 2))
if norm != 0:
    norm_cross_corr /= norm
similarity = np.max(np.abs(norm_cross_corr))
print('similarity:', similarity)



In [ ]:
class Foo:
    def __init__(self):
        pass

    def someother(self, p=None):
        x = '123'
        if p:
            x = p()
        else:
            x = self.somefun()
        print(x)
        pass

    def somefun(self):
        return 'qwe'
    
    def somefun1(self):
        return 'rty'


def somefun2():
    return 'uio'

f = Foo()
f.someother(somefun2)

for k, v in {1: 2}.items():
    print(k, v)

In [ ]:
class Foo:

    def __init__(self, config: dict):

        self.config = config
    
    def greet(self):
        print(f'greetings, {self.config["model"]}')

c = {"model": "1"}
f = Foo(c)
f.greet()
c['model'] = "2"
f.greet()

In [ ]:
def fill_dict(A, T):
    for key, value in T.items():
        if key not in A:
            if isinstance(value, dict):
                A[key] = fill_dict({}, value)
            elif isinstance(value, list):
                A[key] = value[:]
            else:
                A[key] = value
        elif isinstance(A[key], dict) and isinstance(value, dict):
            A[key] = fill_dict(A[key], value)
        elif isinstance(A[key], list) and isinstance(value, list):
            A[key] += [item for item in value if item not in A[key]]
    return A

schema = {
    'rty': '',
    'qweqwe': '',
    's' : {'a':''},
    '1':[{'a':1}, 1]
}

config = {
    's': {'a': 1},
    'qweqwe': 'qweqwe',
    'rty': 1,
    '1': [{'a':2}, 1]
}

fill_dict(config, schema)
'1 2 3'.split(maxsplit=1)

def enumerate_keys(input_dict, parent_key = ''):
    for k, v in input_dict.items():
        current_key = f"{parent_key}.{k}" if parent_key else k
        if isinstance(v, dict):
            yield from enumerate_keys(v, current_key)
        else:
            yield current_key

dict_input = {'a': {'b':1}}

list(enumerate_keys(dict_input))

def set_value(d, key, value):
    keys = key.split('.')
    current_dict = d
    for k in keys[:-1]:
        current_dict[k] = current_dict.get(k, {})
        current_dict = current_dict[k]
    current_dict[keys[-1]] = value
    return d

def get_value(d, key):
    keys = key.split('.')
    current_dict = d
    for k in keys[:-1]:
        current_dict[k] = current_dict.get(k, {})
        current_dict = current_dict[k]
    return current_dict[keys[-1]]


d = {}

set_value(d, '1.2.3.4', '4')
set_value(d, '1.2.3.4', '5')
set_value(d, '1.2', '5')

get_value(d, '1.2')
import json

json.loads(json.dumps({'1': None}))

In [ ]:
class DataObject:
    def __init__(self, data_dict):
        self.__dict__ = data_dict

# Sample dictionary
my_dict = {'name': 'Alice', 'age': 30, 'city': 'New York'}

# Create a data object from the dictionary
data_obj = DataObject(my_dict)

# Access the data object attributes
print(data_obj.name)  # Output: Alice
print(data_obj.age)   # Output: 30
print(data_obj.city)  # Output: New York

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Create a black image
image = np.full((1000, 4000, 3), (255, 255, 255), dtype=np.uint8)

GRID_WIDTH = 250


'''
00x00
0x0x0
00x00
0x0x0
00x00
'''


def draw(image, i, j):
    cv2.line(image, (, 0), (511, 511), (0, 0, 0), 5)

draw(image)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)
plt.axis('on')  # Do not show axis to resemble cv2.imshow
plt.show()